# Worksheet 08

Name: Yuhan Peng<br>
UID: U25596256

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

Trying to figure out what species does this dianasour belong to based on its weight. To do that we need:

$Baye's Rule: Pr(S_i | X_j) = \frac{P(X_i|S_j)P(S_j)}{P(X_i)}$

* $P(S_j)$ is the prior probability of seeing species $S_j$ (that probability would be higher for the Stegosauruses than the Raptors for example). 
* $P(X_i | S_j)$ is the **PDF** of species $S_j$ weights evaluated at weight $X_i$ (seeing a Sauropod that weighs 100 tons is way more likely than seeing a Raptor that weighs 100 tons)

Note: These formulas come from Ruihang Liu's worksheet.


### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[6.142596166438217, 4.080693767380976, 4.510818065225095, 5.400223333821687, 5.050837380039832, 4.185496236270702, 4.583779789938133, 4.281479559679448, 5.26396992356933, 5.651625530322781]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
new_mean = 8
new_stdev = 1

s2 = np.random.normal( new_mean , new_stdev , 10 ).tolist()
print(s2)

[8.576001202720143, 7.215316739681634, 7.816910613569704, 6.921175371130851, 6.736710593953319, 9.916472774161534, 9.138847981168396, 7.334614346760117, 7.350072909231527, 8.522899967750941]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[5.651625530322781, 8.522899967750941, 5.26396992356933, 4.281479559679448, 7.350072909231527, 7.334614346760117, 9.138847981168396, 9.916472774161534, 4.583779789938133, 6.736710593953319]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Means and variances. 

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [9]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[0])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

c:\Users\Yuhan\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Yuhan\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


[5.651625530322781, 5.26396992356933, 4.281479559679448, 4.583779789938133]
[5.651625530322781, 5.26396992356933, 4.281479559679448, 4.583779789938133]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.9452137008774235,  mean_2 = 4.9452137008774235
var_1 = 0.29295017111345867,  var_2 = 0.29295017111345867


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [10]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.651625530322781
probability of observing that point if it came from cluster 0 =  0.07438186190376563
probability of observing that point if it came from cluster 1 =  0.07438186190376563
point =  8.522899967750941
probability of observing that point if it came from cluster 0 =  5.5856317527420656e-33
probability of observing that point if it came from cluster 1 =  5.5856317527420656e-33
point =  5.26396992356933
probability of observing that point if it came from cluster 0 =  0.7534022689623214
probability of observing that point if it came from cluster 1 =  0.7534022689623214
point =  4.281479559679448
probability of observing that point if it came from cluster 0 =  0.10457399714297193
probability of observing that point if it came from cluster 1 =  0.10457399714297193
point =  7.350072909231527
probability of observing that point if it came from cluster 0 =  3.167283623636049e-15
probability of observing that point if it came from cluster 1 =  3.167283623636049e-15
point = 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [14]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * (x[0] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]), sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 6.878047337653553,  mean_2 = 6.878047337653553
var_1 = 203.3974392067478,  var_2 = 16.807605330544373


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [15]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  5.651625530322781
probability of observing that point if it came from cluster 0 =  0.001961357182464156
probability of observing that point if it came from cluster 1 =  0.023672714066689273
point =  8.522899967750941
probability of observing that point if it came from cluster 0 =  0.0019613287030564646
probability of observing that point if it came from cluster 1 =  0.023622428531817292
point =  5.26396992356933
probability of observing that point if it came from cluster 0 =  0.0019613310804760605
probability of observing that point if it came from cluster 1 =  0.023626622233755334
point =  4.281479559679448
probability of observing that point if it came from cluster 0 =  0.0019612330195380606
probability of observing that point if it came from cluster 1 =  0.023454257702252516
point =  7.350072909231527
probability of observing that point if it came from cluster 0 =  0.001961387555674966
probability of observing that point if it came from cluster 1 =  0.02372646060875609
poin

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [16]:
assignments = [[], []]
for x in zip(data, prob_s0_x):
    p = x[1]
    d = x[0]
    if random.random() <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : []
points in cluster 1 : [5.651625530322781, 8.522899967750941, 5.26396992356933, 4.281479559679448, 7.350072909231527, 7.334614346760117, 9.138847981168396, 9.916472774161534, 4.583779789938133, 6.736710593953319]
